In [1]:
import pandas as pd

df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

not_fit = True


In [2]:
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
nltk.download('stopwords')
stop = stopwords.words('english')

def tokenizer_stem_nostop(text):
    porter = PorterStemmer()
    return [porter.stem(w) for w in re.split('\s+', text.strip()) \
            if w not in stop and re.match('[a-zA-Z]+', w)]

[nltk_data] Downloading package stopwords to /Users/AtSu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
import numpy as np
from bs4 import BeautifulSoup
from collections import defaultdict
import src.proc_data as procd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
"""
train_channel = np.zeros((df.values.shape[0], 4), dtype=object)
test_channel = np.zeros((df_test.values.shape[0], 4), dtype=object)
columns = ['entertainment', 'world', 'social-media', 'tech']


for i in range(df['Page content'].values.shape[0]):
    channel = procd.parse_channel(df['Page content'][i])
    if channel in columns:
        train_channel[i, columns.index(channel)] = 1

pd.DataFrame(data=train_channel, columns=columns).to_csv('train_channel.csv', index=False)
        
for i in range(df_test['Page content'].values.shape[0]):
    channel = procd.parse_channel(df_test['Page content'][i])
    if channel in columns:
        test_channel[i, columns.index(channel)] = 1
        
pd.DataFrame(data=test_channel, columns=columns).to_csv('test_channel.csv', index=False)
"""
"""
train_parse = np.zeros((df.values.shape[0], 5),dtype=object)

for i in range(df['Page content'].values.shape[0]):
#for i in range(5):

    train_parse[i, 0], train_parse[i, 1], train_parse[i, 2],  train_parse[i, 3]\
            = procd.parse_html(df['Page content'][i])
    train_parse[i, 4] = len(tokenizer_stem_nostop(train_parse[i, 1]))

pd.DataFrame(data=train_parse, columns=['title', 'content', 'author', 'n_fig', 'n_words']).to_csv('train_b_parsed.csv')

train_parse = np.zeros((df_test.values.shape[0], 5),dtype=object)

#for i in range(5):

for i in range(df_test['Page content'].values.shape[0]):
    train_parse[i, 0], train_parse[i, 1], train_parse[i, 2],  train_parse[i, 3] \
            = procd.parse_html(df_test['Page content'][i])
    train_parse[i, 4] = len(tokenizer_stem_nostop(train_parse[i, 1]))

pd.DataFrame(data=train_parse, columns=['title', 'content', 'author','n_fig', 'n_words']).to_csv('test_b_parsed.csv')
"""

"\ntrain_parse = np.zeros((df.values.shape[0], 5),dtype=object)\n\nfor i in range(df['Page content'].values.shape[0]):\n#for i in range(5):\n\n    train_parse[i, 0], train_parse[i, 1], train_parse[i, 2],  train_parse[i, 3]            = procd.parse_html(df['Page content'][i])\n    train_parse[i, 4] = len(tokenizer_stem_nostop(train_parse[i, 1]))\n\npd.DataFrame(data=train_parse, columns=['title', 'content', 'author', 'n_fig', 'n_words']).to_csv('train_b_parsed.csv')\n\ntrain_parse = np.zeros((df_test.values.shape[0], 5),dtype=object)\n\n#for i in range(5):\n\nfor i in range(df_test['Page content'].values.shape[0]):\n    train_parse[i, 0], train_parse[i, 1], train_parse[i, 2],  train_parse[i, 3]             = procd.parse_html(df_test['Page content'][i])\n    train_parse[i, 4] = len(tokenizer_stem_nostop(train_parse[i, 1]))\n\npd.DataFrame(data=train_parse, columns=['title', 'content', 'author','n_fig', 'n_words']).to_csv('test_b_parsed.csv')\n"

In [4]:
df = pd.read_csv('train_parsed.csv')
df_test = pd.read_csv('test_parsed.csv')


In [7]:
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from time import time

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()
    

n_topics = 5
n_features = 2**10
n_top_words = 20


t0 = time()
tf = CountVectorizer(max_df=0.95, min_df=2,
                        max_features=n_features,
                        tokenizer=tokenizer_stem_nostop)
bow = tf.fit_transform(df['content'])
print('finish fitting %.2f' %(time()-t0))

lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
bow = lda.fit_transform(bow)

tf_feature_names = tf.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

finish fitting 151.16
Topic #0:
app appl use devic phone new imag also googl iphon one user featur design like android screen smartphon make tablet
Topic #1:
compani user facebook new market social year busi servic million ad media googl product use data content site custom mobil
Topic #2:
twitter video imag game show com new year one fan youtub photo time world first tweet play music star also
Topic #3:
like one say get peopl time make work go way want thing think take even would know use look see
Topic #4:
said report state year u accord govern also new citi would one countri peopl time two nation polic imag told



In [9]:
print(bow.shape)
print(bow)

(27643, 5)
[[  9.19352344e-04   9.22218877e-04   9.17085490e-04   3.16070447e-01
    6.81170897e-01]
 [  5.94651522e-01   2.97804894e-01   1.62914643e-03   8.58386614e-02
    2.00757765e-02]
 [  2.18923243e-03   2.18372099e-03   4.39880860e-01   5.53553906e-01
    2.19228086e-03]
 ..., 
 [  8.78441796e-03   8.76005921e-03   8.88997683e-03   9.64736896e-01
    8.82864982e-03]
 [  1.07128165e-03   9.95685035e-01   1.08406483e-03   1.07999799e-03
    1.07962024e-03]
 [  1.21990842e-03   3.10373031e-01   3.80111541e-01   1.05355057e-01
    2.02940463e-01]]


In [46]:
from scipy.sparse import hstack
from sklearn.preprocessing import PolynomialFeatures
train_weekday = pd.read_csv('train_weekday.csv').values
train_time = pd.read_csv('train_time.csv').values
train_channel = pd.read_csv('train_channel.csv').values
poly = PolynomialFeatures(degree=2)
#train_weekday = poly.fit_transform(train_weekday)
#train_time = poly.fit_transform(train_time)
#print(df['n_words'].shape)
#X_train_aug = hstack([train_time.reshape(-1,1), df['n_fig'].reshape(-1,1)])
#X_train_aug = train_weekday
X_train_aug = np.concatenate((train_channel, train_weekday, train_time, bow), axis=1)
X_train_aug = poly.fit_transform(X_train_aug)[:, 1:]
#X_train_aug = train_weekday * train_time
#sc_x = StandardScaler(with_mean=False)
#X_train_aug = sc_x.fit_transform(X_train_aug)
print(X_train_aug.shape)

(27643, 170)


In [47]:
from sklearn.model_selection import train_test_split
# hold out testing set
y_train = df['Popularity']
X_htrain, X_htest, y_htrain, y_htest = train_test_split(X_train_aug, y_train, test_size=0.3, random_state=0)


In [48]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier

SGD = LogisticRegression(C=0.5, random_state=0)
SGD.fit(X_htrain, y_htrain)

pred = SGD.predict(X_htrain)
pred_t = SGD.predict(X_htest)

score_train = metrics.accuracy_score(y_htrain, pred)
score_test = metrics.accuracy_score(y_htest, pred_t)

print("SGD score (train): %f, (test): %f"%(score_train, score_test))
ada = AdaBoostClassifier(base_estimator=SGD, algorithm='SAMME', n_estimators=50)
ada = ada.fit(X_htrain, y_htrain)
pred = ada.predict(X_htrain)
pred_t = ada.predict(X_htest)

score_train = metrics.accuracy_score(y_htrain, pred)
score_test = metrics.accuracy_score(y_htest, pred_t)

print("SGD score (train): %f, (test): %f"%(score_train, score_test))
SGD.fit(X_train_aug, y_train)

pred = SGD.predict(X_htrain)
pred_t = SGD.predict(X_htest)
score_train = metrics.accuracy_score(y_htrain, pred)
score_test = metrics.accuracy_score(y_htest, pred_t)

print("SGD score (train): %f, (test): %f"%(score_train, score_test))
candidates = []
candidates.append({'clf':SGD, 'C':1, 'score':1.0})

ada = AdaBoostClassifier(base_estimator=SGD, algorithm='SAMME', n_estimators=50)
ada = ada.fit(X_train_aug, y_train)
pred = ada.predict(X_htrain)
pred_t = ada.predict(X_htest)

score_train = metrics.accuracy_score(y_htrain, pred)
score_test = metrics.accuracy_score(y_htest, pred_t)

print("SGD score (train): %f, (test): %f"%(score_train, score_test))

candidates.append({'clf' : ada, 'C' : 0.5, 'score':0.55679})

SGD score (train): 0.554470, (test): 0.547450
SGD score (train): 0.549147, (test): 0.546003
SGD score (train): 0.555039, (test): 0.559629
SGD score (train): 0.547855, (test): 0.561920


In [49]:
test_weekday = pd.read_csv('test_weekday.csv').values
test_time = pd.read_csv('test_time.csv').values
test_channel = pd.read_csv('test_channel.csv').values

bow = tf.transform(df_test['content'])
bow = lda.transform(bow)

X_test_con = np.concatenate((test_channel, test_weekday, test_time, bow), axis=1)
X_test_con = poly.transform(X_test_con)[:,1:]
#X_test_con = np.concatenate((test_weekday, test_time), axis=1)
#X_test_con = test_weekday * test_time
#print(X_test_con.shape)
#X_test_std = sc_x.fit_transform(X_test_con)

In [50]:

for cand in candidates:
    predict = cand['clf'].predict(X_test_con)
    output = np.zeros((X_test_con.shape[0], 2), dtype=int)
    output[:, 0] = df_test['Id']
    output[:, 1] = predict
    df_output = pd.DataFrame(data=output, columns=['Id', 'Popularity'])
    df_output.to_csv('test___%f.csv' % (cand['C']), index=False)